## Multilayer Perceptron (MLP)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
%matplotlib inline
%config InlineBackend.figure_format='retina' # pyplot 에서 좋은 더 화질
print ("PyTorch versionL:[%s]." %(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ('device:[%s]'%(device))

## Dataset load

In [ ]:
from torchvision import datasets, transforms
mnist_train = datasets.MNIST(root='./data/', train =True, transform = transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root='./data/', train =False, transform = transforms.ToTensor(), download=True)
print ('mnist_train:\n',mnist_train,'\n')
print ('mnist_test:\n',mnist_train,'\n')
print ('done.')

## Data Iterator

In [ ]:
BATCH_SIZE = 256
train_iter = torch.utils.data.DataLoader(mnist_train,batch_size=BATCH_SIZE,shuffle=True, num_workers=1)
test_iter = torch.utils.data.DataLoader(mnist_train,batch_size=BATCH_SIZE,shuffle=True, num_workers=1)
print('Done')

## Define the MLP model

In [ ]:
class MultiLayerPerceptronClass(nn.Module):
    
    def __init__(self, name='mlp', xdim=784,hdim=256,ydim=10):
        super(MultiLayerPerceptronClass,self).__init__()
        self.name = name
        self.xdim = xdim
        self.hdim = hdim
        self.ydim = ydim
        self.lin_1 = nn.Linear(self.xdim,self.hdim)
        self.lin_2 = nn.Linear(self.hdim,self.ydim)
        self.init_param() # initialize parameters

    def init_param(self):
        nn.init.kaiming_normal_(self.lin_1.weight)
        nn.init.zeros_(self.lin_1.bias)
        nn.init.kaiming_normal_(self.lin_2.weight)
        nn.init.zeros_(self.lin_2.bias)

    def forward(self,x):
        net = x
        net = self.lin_1(net)
        net = F.relu(net)
        net = self.lin_2(net)
        return net

M = MultiLayerPerceptronClass(name='mlp',xdim=784,hdim=256,ydim=10).to(device)
loss = nn.CrossEntropyLoss()
optm = optim.Adam(M.parameters(), lr=1e-3)
print ('Done.')

## Somple Forward Path of the MLP Model

In [ ]:
x_numpy = np.random.rand(2,784)
x_torch = torch.from_numpy(x_numpy).float().to(device)
y_torch = M.forward(x_torch)
y_numpy = y_torch.detach().cpu().numpy()

print ('x_numpy:\n',x_numpy)
print ('x_torch:\n',x_torch)
print ('y_torch:\n',y_torch)
print ('y_numpy:\n',y_numpy)


## Check Parameters

In [ ]:
np.set_printoptions(precision=3)
n_param = 0

for p_idx,(param_name, param) in enumerate(M.named_parameters()):
    param_numpy = param.detach().cpu().numpy()
    n_param += len(param_numpy.reshape(-1))
    print('[%d] name: [%s] shape: [%s].'%(p_idx,param_name,param_numpy.shape))
    print("    val:%s"%(param_numpy.reshape(-1)[:5]))

print("Total number of parameters:[%s]."%(format(n_param, ',d')))

## Evaluation Function

In [ ]:
def func_eval(model, data_iter, device):
    with torch.no_grad():
        model.eval()
        n_total, n_correct = 0,0
        for batch_in, batch_out in data_iter:

            y_trgt = batch_out.to(device)

            model_pred = model(batch_in.view(-1,28*28).to(device))

            _, y_pred = torch.max(model_pred.data,1)
            n_correct += (y_pred == y_trgt).sum().item()
            n_total += batch_in.size(0)

        val_accr = (n_correct/n_total)
        model.train() # back to train mode
    return val_accr
print("Done")

## Initial Evaluation

In [ ]:
train_accr = func_eval(M, train_iter,device)
test_accr = func_eval(M, test_iter, device)
print ('train_accr : [%.3f] test_accr:[%.3f]'%(train_accr,test_accr))

In [ ]:
print (" start training")
M.init_param() # initialize parameters

M.train()

EPOCHS, print_every = 10,1

for epoch in range(EPOCHS):
    loss_val_sum = 0

    for batch_in, batch_out in train_iter:
        # Forward path
        y_pred = M.forward(batch_in.view(-1,28*28).to(device))
        loss_out = loss(y_pred, batch_out.to(device))

        #Updata
        optm.zero_grad()
        loss_out.backward()
        optm.step()

        loss_val_sum += loss_out
    loss_val_avg = loss_val_sum/len(train_iter)

    # print

    if ((epoch%print_every)==0) or (epoch==(EPOCHS-1)):
        train_accr = func_eval(M,train_iter,device)
        test_accr = func_eval(M,test_iter,device)

        print ('epoch:[%d] loss:[%.3f] train_accr:[%.3f] test_accr:[%.3f]'% (epoch,loss_val_avg,train_accr,test_accr))

print ("Done")

In [ ]:
from torchvision import datasets, transforms
mnist_train = datasets.MNIST(root='./data/',train=True,transform=transforms.ToTensor(),download=True)
mnist_test = datasets.MNIST(root='./data/',train=False,transform=transforms.ToTensor(),download=True)

print('mnist_train:\n',mnist_train,'\n')
print('mnist_test:\n',mnist_train,'\n')
print('Done.')